In [ ]:
!pip install --upgrade google-api-python-client

In [1]:
from googleapiclient.discovery import build

In [2]:
youTubeApiKey="xxxxxxxxxxxxxxx"

In [3]:
youtube = build('youtube', 'v3', developerKey=youTubeApiKey)

In [4]:
playlistId = 'PLh2Y_pKOa4UcNJzv9IXUAihhCVv7YkyKQ'
playlistName = 'Universo Data Science'
nextPage_token = None

In [6]:
playlist_videos  = []

while True:
  res = youtube.playlistItems().list(part='snippet', playlistId = playlistId, maxResults=20, pageToken=nextPage_token).execute()
  playlist_videos += res['items']
  
  nextPage_token = res.get('nextPageToken')

  if nextPage_token is None:
    break

In [ ]:
res['items']

In [8]:
videos_ids = list(map(lambda x: x['snippet']['resourceId']['videoId'], playlist_videos))

In [9]:
stats = []

for video_id in videos_ids:
  res = youtube.videos().list(part='statistics', id=video_id).execute()
  stats += res['items']


In [ ]:
stats

In [11]:
videos_title = list(map(lambda x: x['snippet']['title'], playlist_videos))
url_thumbnails = list(map(lambda x: x['snippet']['thumbnails']['high']['url'], playlist_videos))
published_date = list(map(lambda x: str(x['snippet']['publishedAt']), playlist_videos)) #conversion from ISO8601 date format
video_description = list(map(lambda x: x['snippet']['description'], playlist_videos))
videoid = list(map(lambda x: x['snippet']['resourceId']['videoId'], playlist_videos))

In [13]:
liked = list(map(lambda x: int(x['statistics']['likeCount']), stats))
views = list(map(lambda x: int(x['statistics']['viewCount']), stats))
comment = list(map(lambda x: int(x['statistics']['commentCount']), stats))

In [14]:
from datetime import datetime 

extraction_date = [str(datetime.now())]*len(videos_ids)

In [15]:
import pandas as pd

In [ ]:
playlist_df = pd.DataFrame({'title':videos_title,
      'video_id':videoid,
      'video_description':video_description,
      'published_date':published_date,
      'extraction_date':extraction_date,
      'likes':liked,
      'views':views,
      'comment':comment,
      'thumbnail': url_thumbnails})
playlist_df.head(20)

In [28]:
from google.colab import files
playlist_df.to_csv('videos.csv', encoding = 'utf-8-sig')